랜드마크 전체 인식

In [5]:
import cv2 as cv
import numpy as np
import mediapipe as mp
cap = cv.VideoCapture(0)
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(max_num_hands=2, static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)



# 로고 이미지 읽고 크기 조정
logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(3)) # 3은 CAP_PROP_FRAME_WIDTH를 나타내며, 프레임의 가로 길이
frame_height = int(cap.get(4)) # # 4는 CAP_PROP_FRAME_HEIGHT를 나타내며, 프레임의 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV) # 로고 이미지의 흑백 이미지에서 임계값(threshold)을 기준으로 이진화된 마스크(mask)를 생성

# 세로 초록색 선의 좌표 계산
segment_width = frame_width / 14

segment_coordinates = [i * segment_width for i in range(1, 14)]

# 노란색 선의 y 좌표 설정
line_y1 = 280  # 첫 번째 y값
line_y2 = 480  # 두 번째 y값

while True:
    ret, frame = cap.read()

    # 웹캠 화면 가장 아래에 로고 이미지 합성
    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo

    res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

    if res.multi_hand_landmarks: # 프레임에서 손 랜드마크가 감지되었는지를 확인
        for hand_landmarks in res.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)

                if line_y1 <= y <= line_y2:
                    for i in range(len(segment_coordinates)):
                        if x >= segment_coordinates[i] and x <= segment_coordinates[i] + segment_width:
                            cv.circle(frame, (x, y), 5, (0, 255, 255), -1)
                            overlay = frame.copy()
                            cv.rectangle(overlay, (int(segment_coordinates[i]), line_y1),
                                          (int(segment_coordinates[i] + segment_width), line_y2),
                                          (0, 255, 255), -1)
                            alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                            cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                            break

    cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()
